In [287]:
# 没配置过python环境的人第一次运行时将以下pip语句的注释#号去掉
#!pip install math
import math
#!pip install numpy
import numpy as np
#!pip install scipy
from scipy.linalg import pinv #求非方阵矩阵的伪逆矩阵
from scipy import optimize #求解线性规划
#!pip install fractions
from fractions import Fraction
np.set_printoptions(formatter={'all':lambda x: str(Fraction(x).limit_denominator())})
#!pip install copy
import copy
#!pip json
import json

#初始化计算器所需的游戏预置数据



'''
字典recipe_data:配方字典，记录游戏内的配方表，配方表的键值对对应游戏内的某一个配方和它在表内的id

配方字典中，"原料"与"产物"代表生产此物品时的物品净消耗或净产出，以可燃冰高效公式制氢为例：
"原料":{"可燃冰":2},"产物":{石墨烯":2,"氢":1}
若为来源方式为采集，则仅存在产物，以挖掘铁矿为例：
"产物":{"铁矿":1}

配方字典中，"设施"代表生产此物品的工厂类型，包括挖掘设备、抽水机、轨道采集器、冶炼设施等。
公式中仅规定设备类型，以便届时若游戏更新高级化工设备、高级对撞机等设备时容易拓展，也便于与mod的新建筑拓展。

配方字典中，"时间"普遍指完成一趟配方所需的时间，其中较为特殊的是：
挖掘设备的时间规定为2s，小矿机在矿物利用等级为1级时开采1簇矿物时的单位矿物产出时间
采集器的时间定规为1/8s，是矿物利用等级为1级时采集器在面板为1/s的巨星上采集的供电前单位产出时间
抽水机的时间规定为0.8s，是矿物利用等级为1级时单个抽水机的单位产出时间
原油萃取站的时间规定为1s,是矿物利用等级为1级时单个萃取站在面板为1/s的油井上的单位产出时间
分馏塔的时间规定为100s，是让氢以1/s速度过带时的期望单位产出时间
蓄电器（满）的时间未定为300s，是直接接入电网时充满电的时间（直接接入电网，设备倍率为1，使用能量枢纽，则设备倍率为50）

配方字典中，"增产"代表这一配方的可增产性，大致可以看做二进制转十进制
0=0b00代表既不能加速也不能增产
1=0b01代表可以增产但不能加速
2=0b10代表可以加速但不能增产
3=0b11代表可以增产也可以加速
其中，存在一个特例，当使用射线接受站接受光子时用上引力透镜，则引力透镜加速时可以让产出翻倍，
    但不增加引力透镜消耗速度，用9表示这种只能加速但不加倍原料消耗的配方
'''
recipe_data = {
    0: {'原料': {}, '产物': {'木材': 1}, '设施': '轻型工业机甲', '时间': 2, '增产': 0},
    1: {'原料': {}, '产物': {'植物燃料': 1}, '设施': '轻型工业机甲', '时间': 2, '增产': 0},
    2: {'原料': {}, '产物': {'铁矿': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    3: {'原料': {}, '产物': {'铜矿': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    4: {'原料': {}, '产物': {'石矿': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    5: {'原料': {}, '产物': {'煤矿': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    6: {'原料': {}, '产物': {'硅石': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    7: {'原料': {'石矿': 10}, '产物': {'硅石': 1}, '设施': '冶炼设备', '时间': 10, '增产': 3},
    8: {'原料': {}, '产物': {'钛石': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    9: {'原料': {}, '产物': {'水': 1}, '设施': '抽水机', '时间': 0.8, '增产': 0},
    10: {'原料': {}, '产物': {'原油': 1}, '设施': '原油萃取站', '时间': 1, '增产': 0},
    11: {'原料': {}, '产物': {'氢': 1}, '设施': '轨道采集器', '时间': 0.125, '增产': 0},
    12: {'原料': {}, '产物': {'重氢': 1}, '设施': '轨道采集器', '时间': 0.125, '增产': 0},
    13: {'原料': {'氢': 10}, '产物': {'重氢': 5}, '设施': '粒子对撞机', '时间': 2.5, '增产': 2},
    14: {'原料': {'氢': 1}, '产物': {'重氢': 1}, '设施': '分馏塔', '时间': 100, '增产': 2},
    15: {'原料': {'临界光子': 2}, '产物': {'反物质': 2, '氢': 2}, '设施': '粒子对撞机', '时间': 2, '增产': 2},
    16: {'原料': {}, '产物': {'可燃冰': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    17: {'原料': {}, '产物': {'可燃冰': 1}, '设施': '轨道采集器', '时间': 0.125, '增产': 0},
    18: {'原料': {}, '产物': {'金伯利矿石': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    19: {'原料': {}, '产物': {'分形硅石': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    20: {'原料': {}, '产物': {'光栅石': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    21: {'原料': {}, '产物': {'有机晶体': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    22: {'原料': {'塑料': 2, '精炼油': 1, '水': 1}, '产物': {'有机晶体': 1}, '设施': '化工设备', '时间': 6, '增产': 3},
    23: {'原料': {'木材': 20, '植物燃料': 20, '水': 10}, '产物': {'有机晶体': 1}, '设施': '化工设备', '时间': 6, '增产': 3},
    24: {'原料': {}, '产物': {'刺笋结晶': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    25: {'原料': {}, '产物': {'单极磁石': 1}, '设施': '挖掘设备', '时间': 2, '增产': 0},
    26: {'原料': {}, '产物': {'临界光子': 1}, '设施': '射线接收站', '时间': 10, '增产': 0},
    27: {'原料': {'引力透镜': 0.008333}, '产物': {'临界光子': 1}, '设施': '射线接收站', '时间': 5, '增产': 9},
    28: {'原料': {}, '产物': {'硫酸': 1}, '设施': '抽水机', '时间': 0.8, '增产': 0},
    29: {'原料': {'精炼油': 6, '石矿': 8, '水': 4}, '产物': {'硫酸': 4}, '设施': '化工设备', '时间': 6, '增产': 3},
    30: {'原料': {'铁矿': 1}, '产物': {'铁块': 1}, '设施': '冶炼设备', '时间': 1, '增产': 3},
    31: {'原料': {'铜矿': 1}, '产物': {'铜块': 1}, '设施': '冶炼设备', '时间': 1, '增产': 3},
    32: {'原料': {'硅石': 2}, '产物': {'高纯硅块': 1}, '设施': '冶炼设备', '时间': 2, '增产': 3},
    33: {'原料': {'钛石': 2}, '产物': {'钛块': 1}, '设施': '冶炼设备', '时间': 2, '增产': 3},
    34: {'原料': {'石矿': 1}, '产物': {'石材': 1}, '设施': '冶炼设备', '时间': 1, '增产': 3},
    35: {'原料': {'煤矿': 2}, '产物': {'高能石墨': 1}, '设施': '冶炼设备', '时间': 2, '增产': 3},
    36: {'原料': {'精炼油': 1, '氢': 2}, '产物': {'高能石墨': 1, '氢': 3}, '设施': '精炼设备', '时间': 4, '增产': 2},
    37: {'原料': {'石矿': 2}, '产物': {'玻璃': 1}, '设施': '冶炼设备', '时间': 2, '增产': 3},
    38: {'原料': {'铁矿': 1}, '产物': {'磁铁': 1}, '设施': '冶炼设备', '时间': 1.5, '增产': 3},
    39: {'原料': {'金伯利矿石': 1}, '产物': {'金刚石': 2}, '设施': '冶炼设备', '时间': 1.5, '增产': 3},
    40: {'原料': {'高能石墨': 1}, '产物': {'金刚石': 1}, '设施': '冶炼设备', '时间': 2, '增产': 3},
    41: {'原料': {'分形硅石': 1}, '产物': {'晶格硅': 2}, '设施': '制造台', '时间': 1.5, '增产': 3},
    42: {'原料': {'高纯硅块': 1}, '产物': {'晶格硅': 1}, '设施': '冶炼设备', '时间': 2, '增产': 3},
    43: {'原料': {'铁块': 3}, '产物': {'钢材': 1}, '设施': '冶炼设备', '时间': 3, '增产': 3},
    44: {'原料': {'硫酸': 8, '钛块': 4, '钢材': 4}, '产物': {'钛合金': 4}, '设施': '冶炼设备', '时间': 12, '增产': 3},
    45: {'原料': {'原油': 2}, '产物': {'精炼油': 2, '氢': 1}, '设施': '精炼设备', '时间': 4, '增产': 3},
    46: {'原料': {'精炼油': 2, '高能石墨': 1}, '产物': {'塑料': 1}, '设施': '化工设备', '时间': 3, '增产': 3},
    47: {'原料': {'硫酸': 1, '高能石墨': 3}, '产物': {'石墨烯': 2}, '设施': '化工设备', '时间': 3, '增产': 3},
    48: {'原料': {'可燃冰': 2}, '产物': {'石墨烯': 2, '氢': 1}, '设施': '化工设备', '时间': 2, '增产': 3},
    49: {'原料': {'钛块': 1, '石墨烯': 3}, '产物': {'碳纳米管': 2}, '设施': '化工设备', '时间': 4, '增产': 3},
    50: {'原料': {'刺笋结晶': 6}, '产物': {'碳纳米管': 2}, '设施': '化工设备', '时间': 4, '增产': 3},
    51: {'原料': {'铁块': 1}, '产物': {'齿轮': 1}, '设施': '制造台', '时间': 1, '增产': 3},
    52: {'原料': {'玻璃': 3}, '产物': {'棱镜': 2}, '设施': '制造台', '时间': 2, '增产': 3},
    53: {'原料': {'铁块': 2, '铜块': 1}, '产物': {'电路板': 2}, '设施': '制造台', '时间': 1, '增产': 3},
    54: {'原料': {'磁铁': 2, '铜块': 1}, '产物': {'磁线圈': 2}, '设施': '制造台', '时间': 1, '增产': 3},
    55: {'原料': {'高纯硅块': 2, '铜块': 1}, '产物': {'微晶元件': 1}, '设施': '制造台', '时间': 2, '增产': 3},
    56: {'原料': {'钛块': 3, '有机晶体': 1}, '产物': {'钛晶石': 1}, '设施': '制造台', '时间': 4, '增产': 3},
    57: {'原料': {'磁线圈': 4, '棱镜': 2}, '产物': {'电浆激发器': 1}, '设施': '制造台', '时间': 2, '增产': 3},
    58: {'原料': {'磁线圈': 1, '铁块': 2, '齿轮': 1}, '产物': {'电动机': 1}, '设施': '制造台', '时间': 2, '增产': 3},
    59: {'原料': {'磁线圈': 2, '电动机': 2}, '产物': {'电磁涡轮': 1}, '设施': '制造台', '时间': 2, '增产': 3},
    60: {'原料': {'电路板': 2, '微晶元件': 2}, '产物': {'处理器': 1}, '设施': '制造台', '时间': 3, '增产': 3},
    61: {'原料': {'碳纳米管': 2, '晶格硅': 2, '塑料': 1}, '产物': {'粒子宽带': 1}, '设施': '制造台', '时间': 8, '增产': 3},
    62: {'原料': {'碳纳米管': 4, '高纯硅块': 1, '钛合金': 1}, '产物': {'框架材料': 1}, '设施': '制造台', '时间': 6, '增产': 3},
    63: {'原料': {'电路板': 1, '棱镜': 2}, '产物': {'光子合并器': 1}, '设施': '制造台', '时间': 3, '增产': 3},
    64: {'原料': {'电路板': 1, '光栅石': 1}, '产物': {'光子合并器': 1}, '设施': '制造台', '时间': 3, '增产': 3},
    65: {'原料': {'电磁涡轮': 2, '铜块': 2, '石墨烯': 2}, '产物': {'粒子容器': 1}, '设施': '制造台', '时间': 4, '增产': 3},
    66: {'原料': {'单极磁石': 10, '铜块': 2}, '产物': {'粒子容器': 1}, '设施': '制造台', '时间': 4, '增产': 3},
    67: {'原料': {'电磁涡轮': 2, '磁铁': 3, '高能石墨': 1}, '产物': {'超级磁场环': 1}, '设施': '制造台', '时间': 3, '增产': 3},
    68: {'原料': {'金刚石': 4, '奇异物质': 1}, '产物': {'引力透镜': 1}, '设施': '制造台', '时间': 6, '增产': 3},
    69: {'原料': {'玻璃': 2, '钛块': 2, '水': 2}, '产物': {'钛化玻璃': 2}, '设施': '制造台', '时间': 5, '增产': 3},
    70: {'原料': {'钛晶石': 1, '石墨烯': 2, '氢': 12}, '产物': {'卡西米尔晶体': 1}, '设施': '制造台', '时间': 4, '增产': 3},
    71: {'原料': {'光栅石': 8, '石墨烯': 2, '氢': 12}, '产物': {'卡西米尔晶体': 1}, '设施': '制造台', '时间': 4, '增产': 3},
    72: {'原料': {'卡西米尔晶体': 1, '钛化玻璃': 2}, '产物': {'位面过滤器': 1}, '设施': '制造台', '时间': 12, '增产': 3},
    73: {'原料': {'处理器': 2, '位面过滤器': 2}, '产物': {'量子芯片': 1}, '设施': '制造台', '时间': 6, '增产': 3},
    74: {'原料': {'处理器': 1, '粒子容器': 1}, '产物': {'湮灭约束球': 1}, '设施': '制造台', '时间': 20, '增产': 3},
    75: {'原料': {'铁块': 2, '粒子容器': 2, '重氢': 10}, '产物': {'奇异物质': 1}, '设施': '粒子对撞机', '时间': 8, '增产': 3},
    76: {'原料': {'煤矿': 1}, '产物': {'增产剂MK.Ⅰ': 1}, '设施': '制造台', '时间': 0.5, '增产': 3},
    77: {'原料': {'金刚石': 1, '增产剂MK.Ⅰ': 2}, '产物': {'增产剂MK.Ⅱ': 1}, '设施': '制造台', '时间': 1, '增产': 3},
    78: {'原料': {'碳纳米管': 1, '增产剂MK.Ⅱ': 2}, '产物': {'增产剂MK.Ⅲ': 1}, '设施': '制造台', '时间': 2, '增产': 3},
    79: {'原料': {'钛块': 1, '氢': 10}, '产物': {'液氢燃料棒': 2}, '设施': '制造台', '时间': 6, '增产': 3},
    80: {'原料': {'钛合金': 1, '重氢': 20, '超级磁场环': 1}, '产物': {'氘核燃料棒': 2}, '设施': '制造台', '时间': 12, '增产': 3},
    81: {'原料': {'钛合金': 1, '氢': 12, '反物质': 12, '湮灭约束球': 1}, '产物': {'反物质燃料棒': 2}, '设施': '制造台', '时间': 24,
         '增产': 2},
    82: {'原料': {'引力透镜': 1}, '产物': {'空间翘曲器': 1}, '设施': '制造台', '时间': 10, '增产': 3},
    83: {'原料': {'引力矩阵': 1}, '产物': {'空间翘曲器': 8}, '设施': '制造台', '时间': 10, '增产': 3},
    84: {'原料': {'石墨烯': 1, '光子合并器': 1}, '产物': {'太阳帆': 2}, '设施': '制造台', '时间': 4, '增产': 3},
    85: {'原料': {'框架材料': 3, '处理器': 3, '太阳帆': 3}, '产物': {'戴森球组件': 1}, '设施': '制造台', '时间': 8, '增产': 3},
    86: {'原料': {'量子芯片': 2, '氘核燃料棒': 4, '戴森球组件': 2}, '产物': {'小型运载火箭': 1}, '设施': '制造台', '时间': 6, '增产': 3},
    87: {'原料': {'磁线圈': 1, '电路板': 1}, '产物': {'电磁矩阵': 1}, '设施': '科研设备', '时间': 3, '增产': 3},
    88: {'原料': {'高能石墨': 2, '氢': 2}, '产物': {'能量矩阵': 1}, '设施': '科研设备', '时间': 6, '增产': 3},
    89: {'原料': {'金刚石': 1, '钛晶石': 1}, '产物': {'结构矩阵': 1}, '设施': '科研设备', '时间': 8, '增产': 3},
    90: {'原料': {'处理器': 2, '粒子宽带': 1}, '产物': {'信息矩阵': 1}, '设施': '科研设备', '时间': 10, '增产': 3},
    91: {'原料': {'引力透镜': 1, '量子芯片': 1}, '产物': {'引力矩阵': 2}, '设施': '科研设备', '时间': 24, '增产': 3},
    92: {'原料': {'电磁矩阵': 1, '能量矩阵': 1, '结构矩阵': 1, '信息矩阵': 1, '引力矩阵': 1, '反物质': 1}, '产物': {'宇宙矩阵': 1},
         '设施': '科研设备', '时间': 15, '增产': 3},
    93: {'原料': {'钢材': 2, '铜块': 3}, '产物': {'推进器': 1}, '设施': '制造台', '时间': 4, '增产': 3},
    94: {'原料': {'钛合金': 5, '电磁涡轮': 5}, '产物': {'加力推进器': 1}, '设施': '制造台', '时间': 6, '增产': 3},
    95: {'原料': {'铁块': 5, '处理器': 2, '推进器': 2}, '产物': {'物流运输机': 1}, '设施': '制造台', '时间': 4, '增产': 2},
    96: {'原料': {'钛合金': 10, '处理器': 10, '加力推进器': 2}, '产物': {'星际物流运输船': 1}, '设施': '制造台', '时间': 6, '增产': 2},
    97: {'原料': {'钢材': 1, '石材': 3}, '产物': {'地基': 1}, '设施': '制造台', '时间': 1, '增产': 3},
    98: {'原料': {'铁块': 2, '磁线圈': 1}, '产物': {'电力感应塔': 1}, '设施': '制造台', '时间': 1, '增产': 3},
    99: {'原料': {'电力感应塔': 1, '电浆激发器': 3}, '产物': {'无线输电塔': 1}, '设施': '制造台', '时间': 3, '增产': 2},
    100: {'原料': {'无线输电塔': 1, '超级磁场环': 10, '框架材料': 2}, '产物': {'卫星配电站': 1}, '设施': '制造台', '时间': 5, '增产': 2},
    101: {'原料': {'铁块': 6, '齿轮': 1, '磁线圈': 3}, '产物': {'风力涡轮机': 1}, '设施': '制造台', '时间': 4, '增产': 3},
    102: {'原料': {'铁块': 10, '石材': 4, '齿轮': 4, '磁线圈': 4}, '产物': {'火力发电机': 1}, '设施': '制造台', '时间': 5, '增产': 3},
    103: {'原料': {'高纯硅块': 10, '铜块': 10, '电路板': 5}, '产物': {'太阳能板': 1}, '设施': '制造台', '时间': 6, '增产': 3},
    104: {'原料': {'铁块': 6, '超级磁场环': 1, '晶格硅': 6}, '产物': {'蓄电器': 1}, '设施': '制造台', '时间': 5, '增产': 3},
    105: {'原料': {'蓄电器': 1}, '产物': {'蓄电器（满）': 1}, '设施': '充电设备', '时间': 300, '增产': 0},
    106: {'原料': {'钢材': 15, '超级磁场环': 1, '铜块': 20, '光子合并器': 4}, '产物': {'地热发电站': 1}, '设施': '制造台', '时间': 6,
          '增产': 3},
    107: {'原料': {'钛合金': 12, '超级磁场环': 10, '碳纳米管': 8, '处理器': 4}, '产物': {'微型聚变发电站': 1}, '设施': '制造台', '时间': 10,
          '增产': 3},
    108: {'原料': {'钢材': 40, '钛合金': 40, '处理器': 40, '粒子容器': 8}, '产物': {'能量枢纽': 1}, '设施': '制造台', '时间': 15,
          '增产': 3},
    109: {'原料': {'钢材': 20, '高纯硅块': 20, '处理器': 5, '光子合并器': 10, '超级磁场环': 20}, '产物': {'射线接收站': 1}, '设施': '制造台',
          '时间': 8, '增产': 3},
    110: {'原料': {'钛合金': 20, '框架材料': 20, '湮灭约束球': 10, '量子芯片': 10}, '产物': {'人造恒星': 1}, '设施': '制造台', '时间': 30,
          '增产': 3},
    111: {'原料': {'铁块': 2, '齿轮': 1}, '产物': {'传送带': 3}, '设施': '制造台', '时间': 1, '增产': 3},
    112: {'原料': {'传送带': 3, '电磁涡轮': 1}, '产物': {'高速传送带': 3}, '设施': '制造台', '时间': 1, '增产': 2},
    113: {'原料': {'高速传送带': 3, '超级磁场环': 1, '石墨烯': 1}, '产物': {'极速传送带': 3}, '设施': '制造台', '时间': 1, '增产': 2},
    114: {'原料': {'铁块': 3, '齿轮': 2, '电路板': 1}, '产物': {'四向分流器': 1}, '设施': '制造台', '时间': 2, '增产': 3},
    115: {'原料': {'钢材': 3, '超级磁场环': 1, '齿轮': 4, '处理器': 2}, '产物': {'自动集装机': 1}, '设施': '制造台', '时间': 4, '增产': 3},
    116: {'原料': {'铁块': 3, '电路板': 2, '齿轮': 2, '玻璃': 1}, '产物': {'流速监测器': 1}, '设施': '制造台', '时间': 2, '增产': 3},
    117: {'原料': {'铁块': 4, '石材': 4}, '产物': {'小型储物仓': 1}, '设施': '制造台', '时间': 2, '增产': 3},
    118: {'原料': {'钢材': 8, '石材': 8}, '产物': {'大型储物仓': 1}, '设施': '制造台', '时间': 4, '增产': 3},
    119: {'原料': {'铁块': 8, '石材': 4, '玻璃': 4}, '产物': {'储液罐': 1}, '设施': '制造台', '时间': 2, '增产': 3},
    120: {'原料': {'钛块': 40, '钢材': 40, '处理器': 40, '粒子容器': 20}, '产物': {'行星内物流运输站': 1}, '设施': '制造台', '时间': 20,
          '增产': 3},
    121: {'原料': {'钛合金': 40, '粒子容器': 20, '行星内物流运输站': 1}, '产物': {'星际物流运输站': 1}, '设施': '制造台', '时间': 30,
          '增产': 2},
    122: {'原料': {'超级磁场环': 50, '加力推进器': 20, '蓄电器（满）': 20, '星际物流运输站': 1}, '产物': {'轨道采集器': 1}, '设施': '制造台',
          '时间': 30, '增产': 2},
    123: {'原料': {'铁块': 1, '电路板': 1}, '产物': {'分拣器': 1}, '设施': '制造台', '时间': 1, '增产': 3},
    124: {'原料': {'分拣器': 2, '电动机': 1}, '产物': {'高速分拣器': 2}, '设施': '制造台', '时间': 1, '增产': 2},
    125: {'原料': {'高速分拣器': 2, '电磁涡轮': 1}, '产物': {'极速分拣器': 2}, '设施': '制造台', '时间': 1, '增产': 2},
    126: {'原料': {'铁块': 4, '电路板': 2, '磁线圈': 2, '齿轮': 2}, '产物': {'采矿机': 1}, '设施': '制造台', '时间': 3, '增产': 3},
    127: {'原料': {'钛合金': 20, '框架材料': 10, '超级磁场环': 10, '量子芯片': 2, '光栅石': 40}, '产物': {'大型采矿机': 1}, '设施': '制造台',
          '时间': 20, '增产': 3},
    128: {'原料': {'铁块': 8, '电路板': 2, '电动机': 4, '石材': 4}, '产物': {'抽水机': 1}, '设施': '制造台', '时间': 4, '增产': 3},
    129: {'原料': {'钢材': 12, '电路板': 6, '石材': 12, '电浆激发器': 4}, '产物': {'原油萃取站': 1}, '设施': '制造台', '时间': 8,
          '增产': 3},
    130: {'原料': {'钢材': 10, '电路板': 6, '石材': 10, '电浆激发器': 6}, '产物': {'原油精炼厂': 1}, '设施': '制造台', '时间': 6,
          '增产': 3},
    131: {'原料': {'钢材': 8, '处理器': 1, '石材': 4, '玻璃': 4}, '产物': {'分馏塔': 1}, '设施': '制造台', '时间': 3, '增产': 3},
    132: {'原料': {'钛合金': 20, '处理器': 8, '框架材料': 20, '超级磁场环': 50, '石墨烯': 10}, '产物': {'微型粒子对撞机': 1}, '设施': '制造台',
          '时间': 15, '增产': 3},
    133: {'原料': {'钢材': 20, '处理器': 5, '齿轮': 20, '超级磁场环': 10}, '产物': {'电磁轨道弹射器': 1}, '设施': '制造台', '时间': 6,
          '增产': 3},
    134: {'原料': {'钛合金': 80, '框架材料': 30, '引力透镜': 20, '量子芯片': 10}, '产物': {'垂直发射井': 1}, '设施': '制造台', '时间': 30,
          '增产': 3},
    135: {'原料': {'铁块': 4, '齿轮': 8, '电路板': 4}, '产物': {'制造台MK.Ⅰ': 1}, '设施': '制造台', '时间': 0.5, '增产': 3},
    136: {'原料': {'处理器': 4, '石墨烯': 8, '制造台MK.Ⅰ': 1}, '产物': {'制造台MK.Ⅱ': 1}, '设施': '制造台', '时间': 3, '增产': 2},
    137: {'原料': {'量子芯片': 2, '粒子宽带': 8, '制造台MK.Ⅱ': 1}, '产物': {'制造台MK.Ⅲ': 1}, '设施': '制造台', '时间': 4, '增产': 2},
    138: {'原料': {'铁块': 4, '电路板': 4, '磁线圈': 2, '石材': 2}, '产物': {'电弧熔炉': 1}, '设施': '制造台', '时间': 3, '增产': 3},
    139: {'原料': {'电弧熔炉': 1, '框架材料': 5, '位面过滤器': 4, '单极磁石': 15}, '产物': {'位面熔炉': 1}, '设施': '制造台', '时间': 5,
          '增产': 2},
    140: {'原料': {'钢材': 4, '电浆激发器': 2, '电路板': 2, '微晶元件': 2}, '产物': {'喷涂机': 1}, '设施': '制造台', '时间': 3, '增产': 3},
    141: {'原料': {'钢材': 8, '石材': 8, '电路板': 2, '玻璃': 8}, '产物': {'化工厂': 1}, '设施': '制造台', '时间': 5, '增产': 3},
    142: {'原料': {'铁块': 8, '玻璃': 4, '电路板': 4, '磁线圈': 4}, '产物': {'研究站': 1}, '设施': '制造台', '时间': 3, '增产': 3},
    143: {'原料': {'精炼油': 2, '氢': 1, '煤矿': 1}, '产物': {'精炼油': 3}, '设施': '精炼设备', '时间': 4, '增产': 2}
}


'''
列表proliferator_data记录各级增产剂的增产、加速效果与喷涂次数，其中proliferator_data[0]代表0级增产剂（不增产），依次往上递增
'''
proliferator_data = [
    {'增产剂名称':'无','增产效果':1,'加速效果':1,'耗电变化':1,'基础喷涂次数':0,'被喷涂后次数':0,'喷涂者等级':0},
    {'增产剂名称':'增产剂MK.Ⅰ','增产效果':1.125,'加速效果':1.25,'耗电变化':1.3,'基础喷涂次数':12,'被喷涂后次数':12,'喷涂者等级':0},
    {'增产剂名称':'增产剂MK.Ⅱ','增产效果':1.2,'加速效果':1.5,'耗电变化':1.7,'基础喷涂次数':24,'被喷涂后次数':24,'喷涂者等级':0},
    {'增产剂名称':'增产剂MK.Ⅲ','增产效果':1.25,'加速效果':2,'耗电变化':2.5,'基础喷涂次数':60,'被喷涂后次数':60,'喷涂者等级':0},
]


'''
item_data用于记录物品信息，表示某一物品的id以及其可能来源于哪些配方,表现为：
'物品名'：[物品id,物品来源配方1,物品来源配方2...]
'''
item_data = {}
item_num = 0
for i in recipe_data:
    for j in recipe_data[i]['产物']:
        if j in item_data:
            item_data[j].append(i)
        else:
            item_data[j] = [item_num]
            item_data[j].append(i)
            item_num += 1


'''
列表recipe_choices用于记录配方选取方法，其中recipe_choices[i]表示id为i的物品的来源配方选取
'来源配方':表示选取item_data中对应下标的id的哪一个来源配方
'增产剂等级':表示选取什么等级的增产剂（0为不使用，1、2、3分别对应增产剂MK.Ⅰ、Ⅱ、Ⅲ）
'增产模式':表示选取何种增产模式（0为不增产，1为增产，2为加速）
'物品名':表示此id的物品名字
'''
recipe_choices=[{'来源配方':1, '增产剂等级':3, '增产模式':0, '物品名':i} for i in range(0,len(item_data))]
for i in item_data:
    recipe_choices[item_data[i][0]]['物品名'] = i
        

'''
列表needy_list用于记录所需的净产出物品列表（就是之前版本量化计算器最顶端的需求列表）
其中needy_list[i]表示为id为i的物品的净产出需求,默认全为0
'''
needy_list = np.zeros(len(item_data))



In [636]:

# 更改设置部分代码


'''
get_recipe_table(item_name):获取物品的所有来源配方
输入：所需查询的物品名称
输出：可合成此物品的所有配方
例：
get_recipe_table('氢')

out：[{配方一信息},{配方二信息}...]
'''
def get_recipe_table(item_name):
    recipes = []
    for i in range(1,len(item_data[item_name])):
        recipes.append(copy.deepcopy(recipe_data[item_data[item_name][i]]))
    return recipes


'''
proliferator_plus(proliferator,level):更改增产剂被喷涂情况
输入：所需更改的增产剂名称，喷涂等级（注意并不是点数3级增产剂的等级是3而非4）
作用：此函数可以更改proliferator_data中的信息已使得之后创建配方矩阵时相应的数据得以更改
例：
proliferator_plus('增产剂MK.Ⅲ',3)

效果：将增产剂MK.Ⅲ设置为先自喷涂再使用
'''
def proliferator_plus(proliferator,level):
    for i in proliferator_data:
        if i['增产剂名称'] == proliferator:
            i['被喷涂后次数'] = int(i['基础喷涂次数']*proliferator_data[level]['增产效果'])
            i['喷涂者等级'] = level


'''
change_recipe(item_name,recipe_id,proliferator,product_mode):更改配方选取表
输入：(所需更改的产物名称，
    欲选用的此产物的第几个配方，
    使用几级增产剂（0为不使用，1、2、3分别对应增产剂MK.Ⅰ、Ⅱ、Ⅲ），
    增产模式（0为不增产，1为增产，2为加速）
    )
效果：改变物品的生产方式以改变配方矩阵
例：
change_recipe('氢',3,3,2)

效果：将氢的获取方式设置为来源表中的第3个方式（X裂解）,使用3级增产剂，选择增产模式
'''
def change_recipe(item_name,recipe_id,proliferator,product_mode):
    recipe_choices[item_data[item_name][0]]['来源配方'] = recipe_id
    recipe_choices[item_data[item_name][0]]['增产剂等级'] = proliferator
    if product_mode == 0:
            recipe_choices[item_data[item_name][0]]['增产模式'] = product_mode #应该所有配方都有不适用增产剂的生产模式吧
    elif (recipe_data[item_data[item_name][recipe_id]]['增产'] >> (product_mode - 1) & 1) == 1:
        recipe_choices[item_data[item_name][0]]['增产模式'] = product_mode #配方中二进制记录各模式的是否可以选取情况，这边就是取对应位的数字判断是否为1


'''
change_proliferator_all(proliferator):一键更改所有配方采用的增产剂等级
输入：所需采用的增产剂等级（0为不使用，1、2、3分别对应增产剂MK.Ⅰ、Ⅱ、Ⅲ）
效果：将所有配方使用的增产剂皆改为这个等级的增产剂
例：
change_proliferator_all(3)

效果：将所有配方用到的增产剂都换成3级增产剂（不会改变3级增产剂是否自喷涂或被其它增产剂喷涂的设置）
'''
def change_proliferator_all(proliferator):
    for i in recipe_choices:
        i['增产剂等级'] = proliferator
        

'''
change_product_mode_all(product_mode):一键更改所有配方的增产模式
输入：所需增产模式（0为不增产，1为增产，2为加速）
效果：将所有配方的增产模式替换为输入值（若配方无对应增产模式，则不进行改变增产模式的操作）

例：
change_product_mode_all(2)
change_product_mode_all(1)

效果：将所有可以加速的配方改为加速模式，然后将所有可以增产的配方改为增产模式
'''
def change_product_mode_all(product_mode):
    for i in recipe_choices:
        if product_mode == 0:
            i['增产模式'] = product_mode #应该所有配方都有不适用增产剂的生产模式吧
        elif (recipe_data[item_data[i['物品名']][i['来源配方']]]['增产'] >> (product_mode - 1) & 1) == 1:
            i['增产模式'] = product_mode #配方中二进制记录各模式的是否可以选取情况，这边就是取对应位的数字判断是否为1


'''
save_data:将配方设置等数据保存为指定路径下的json文件
输入：欲保存位置的路径

例：save_data('创世之书mod-方案1')

效果：将recipe_data(配方字典)，proliferator_data(增产剂效果字典)，recipe_choices(生产方式决策列表)，
        item_data(物品字典)，needy_list(需求列表)写入文件
'''
def save_data(path):
    data = {}
    data['recipe_data'] = recipe_data
    data['proliferator_data'] = proliferator_data
    data['recipe_choices'] = recipe_choices
    data['item_data'] = item_data
    data['needy_list'] = needy_list.tolist()
    with open(path + '.json','w',encoding='utf8')as fp:
        json_dict = json.dumps(data)
        fp.write(json_dict)


'''
load_data:读取指定路径下的json文件
输入：欲读取位置的路径

例：save_data('创世之书mod-方案1')

效果：将recipe_data(配方字典)，proliferator_data(增产剂效果字典)，recipe_choices(生产方式决策列表)，
        item_data(物品字典)，needy_list(需求列表)从文件读入程序
'''
def load_data(path):
    data = json.load(open(path+'.json','r',encoding="utf-8"))
    recipe_data = data['recipe_data']
    proliferator_data = data['proliferator_data']
    recipe_choices = data['recipe_choices']
    item_data = data['item_data']
    needy_list = np.array(data['needy_list'])


'''
set_needy_list:设置某一物品需求量
输入：
item：物品名字
num：需求数量

例：set_needy_list('氢',500)

效果：将needy_list中的氢需求量改为500
'''
def set_needy_list(item,num):
    needy_list[item_data[item][0]] = num


'''
debug自用，看看哪里出了问题，用于矩阵中数据足够大的部分所指的物品和对应值
'''
def show_require(require_list):
    v_list = {}
    for i in range(0,len(require_list)):
        if abs(require_list[i]) > 1e-6:
            v_list[list(item_data)[i]] = require_list[i]
    print(v_list)

In [644]:

# 计算部分

'''
price_mat_calculate():
    计算代价矩阵，代价矩阵中的第i行第j列代表想要完成1趟id为i的物品的获取来源配方时需要的id为j的物品来源配方的物品数
输出：代价矩阵与多来源产物表及相关的配方列表
'''
def price_mat_calculate():
    recipe_mat = np.zeros((len(item_data),len(item_data)))
    multi_item_list = np.zeros(len(item_data))
    multi_recipe_list = []
    
    proliferate_price = np.eye(len(proliferator_data)-1) #根据增产剂用时喷涂情况更改增产剂喷涂成本
    for i in range(1,len(proliferator_data)):
        if proliferator_data[i]['喷涂者等级'] != 0:
            proliferate_price[i-1][proliferator_data[i]['喷涂者等级']-1] -= 1 / proliferator_data[proliferator_data[i]['喷涂者等级']]['被喷涂后次数']
    proliferate_price = np.array(np.mat(proliferate_price).I) #求出每一个增产剂的互喷涂成本
    for i in range(0,len(proliferator_data)-1):
        for j in range(0,len(proliferator_data)-1):
            proliferate_price[i][j] /= proliferator_data[i+1]['被喷涂后次数'] #求出各个等级的每一次喷涂成本
    
    for item in item_data:
        recipe_choice = recipe_choices[item_data[item][0]]
        recipe = recipe_data[item_data[item][recipe_choice['来源配方']]]
        # 单趟配方总消耗产出计算
        recipe_row = np.zeros(len(item_data))
        
        if recipe_choice['增产模式'] == 0 or recipe_choice['增产剂等级'] == 0: #该配方不使用增产剂
            for require in recipe['原料']:
                recipe_row[item_data[require][0]] -= recipe['原料'][require]
            for product in recipe['产物']:
                recipe_row[item_data[product][0]] += recipe['产物'][product]
        elif recipe_choice['增产模式'] == 1: #增产模式为增产
            num1 = 0
            for require in recipe['原料']:
                num1 += recipe['原料'][require] #num1用于记录完成一趟配方需要的原料数，用以记录须消耗多少喷涂的增产剂
                recipe_row[item_data[require][0]] -= recipe['原料'][require]
            for product in recipe['产物']:
                recipe_row[item_data[product][0]] += recipe['产物'][product]*proliferator_data[recipe_choice['增产剂等级']]['增产效果']
            for i in range(0,len(proliferator_data)-1):
                recipe_row[item_data[proliferator_data[i+1]['增产剂名称']][0]] -= num1 * proliferate_price[recipe_choice['增产剂等级']-1][i]
        elif recipe_choice['增产模式'] == 2: #增产模式为加速
            num1 = 0
            speed = proliferator_data[recipe_choice['增产剂等级']]['加速效果']
            for require in recipe['原料']:
                num1 += speed * recipe['原料'][require] #num1用于记录完成一趟配方需要的原料数，用以记录须消耗多少喷涂的增产剂
                recipe_row[item_data[require][0]] -= speed * recipe['原料'][require]
            for product in recipe['产物']:
                recipe_row[item_data[product][0]] += speed * recipe['产物'][product]
            for i in range(0,len(proliferator_data)-1):
                recipe_row[item_data[proliferator_data[i+1]['增产剂名称']][0]] -= num1 * proliferate_price[recipe_choice['增产剂等级']-1][i]

        # 判断产物来源是否单一，是否有必要进行线性规划求最优解
        flag1 = 0
        count = 0
        for product in recipe['产物']:
            if multi_item_list[item_data[product][0]] == -1:
                flag1 = 1
            elif multi_item_list[item_data[product][0]] != 0 and product != item:
                multi_recipe_list.append(copy.deepcopy(recipe_mat[int(multi_item_list[item_data[product][0]])]))#之前生产过这个产物的配方也涉及多来源计算
                for product1 in range(0,len(item_data)):
                    if recipe_mat[item_data[product][0]][product1] > 1e-6:
                        multi_item_list[product1] = -1
                        if product1 == item_data[product][0]:
                            recipe_mat[item_data[product][0]][product1] = 1
                        else:
                            recipe_mat[item_data[product][0]][product1] = 0
                    else:
                        recipe_mat[item_data[product][0]][product1] = 0
                flag1 = 1
        if flag1 == 0: #该配方目前不涉及多来源产物
            recipe_mat[item_data[item][0]] = copy.deepcopy(recipe_row)
            for product in recipe['产物']:
                multi_item_list[item_data[product][0]] = item_data[item][0]
        else: #此配方涉及多来源产物的计算
            multi_recipe_list.append(copy.deepcopy(recipe_row))
            recipe_mat[item_data[item][0]] = np.zeros(len(item_data))
            recipe_mat[item_data[item][0]][[item_data[item][0]]] = 1
            for product in recipe['产物']:
                multi_item_list[item_data[product][0]] = -1
       
    price_mat = np.mat(recipe_mat).I # 计算代价矩阵，即单产物配方矩阵的逆矩阵
    multi_item_list1 = []
    for i in range(0,len(item_data)):
        if multi_item_list[i] == -1:
            multi_item_list1.append(i)
    for i in range(0,len(multi_item_list1)): # 将多产物配方中对应产物的系数变为1，方便后面线性规划计算成本
        num1 = copy.deepcopy(multi_recipe_list[i][multi_item_list1[i]])
        for j in range(0,len(item_data)):
            multi_recipe_list[i][j] = multi_recipe_list[i][j]/num1
    for i in range(0,len(multi_item_list1)): # 复位配方矩阵
        recipe_mat[multi_item_list1[i]] = copy.deepcopy(multi_recipe_list[i])
    for i in range(0,len(item_data)): #消除price_mat中接近0的数字的浮点误差
        for j in range(0,len(item_data)):
            if abs(price_mat[i,j]) < 1e-6:
                price_mat[i,j] = 0
    return price_mat,multi_item_list1,multi_recipe_list,recipe_mat


'''
require_calculate：总成本即毛产出计算函数，量化计算器关于物品方面的结果的最终输出
输入：
    needy_list：需求列表，即需求的物品净产出
    price_mat：代价矩阵，由函数recipe_mat_calculat()得出
    multi_item_list：记录需要进行线性规划的多来源物品列表,记录多来源物品的id
    multi_recipe_list：在此生产模式下可能有多重来源的物品涉及到的配方
输出：总成本即毛产出列表
例：（在所有配方均不使用增产剂的情况下）
price_mat,multi_item_list,multi_recipe_list = price_mat_calculate()
needy_list = [0,0,0,...2(此时下标为电路板id),0,0,0...,0]
print(require_calculate(needy_list,price_mat,multi_item_list,multi_recipe_list))

结果：[0...2(id为铁矿),1(id为铜矿)...2(id为铁块),1(id为铜块)...2(id为电路板),0,0...,0]
'''
def require_calculate(needy_list,price_mat,multi_item_list,multi_recipe_list1):
    multi_recipe_list = copy.deepcopy(multi_recipe_list1)
    require_list = np.array(np.mat(needy_list) * price_mat)[0]
    multi_price = []
    multi_needy_list = [] 
    for i in range(0,len(multi_recipe_list)):#将需要线性规划的多产物配方的需求划分开来
        multi_needy_list.append(copy.copy(require_list[multi_item_list[i]]))
        require_list[multi_item_list[i]] = 0
    for i in range(0,len(multi_recipe_list)):
        price = 0
        multi_recipe_list[i] =  np.array(np.mat(multi_recipe_list[i]) * price_mat)[0]
        recipe_time = recipe_data[item_data[list(item_data)[multi_item_list[i]]][recipe_choices[multi_item_list[i]]['来源配方']]]['时间']
        for j in range(0,len(item_data)):
            if multi_recipe_list[i][j] < -1e-6:
                price -= multi_recipe_list[i][j]/recipe_time
                '''此处应计算生产此物品的历史总成本，根据用户需求应该可以选择是否计算耗电量、如何计算等。
                此项程序体量较为庞大，日后再和建筑计算总计占地、电力成本计算模块一同实现，此处暂时默认为同产能需求的历史生产物品总数'''
        if price < 1e-2: #防止有些人成本设置出现差错或因浮点误差使0值变为负数的情况发生导致线性规划结果不稳定，将不够大的成本设为一个小正数
            price = 1e-2
        if multi_needy_list[i] == 0:
            price = 1e5
        multi_price.append(copy.copy(price))
        
    c = multi_price
    A = []
    for i in range(0,len(multi_recipe_list)):
        A.append([-multi_recipe_list[j][multi_item_list[i]] for j in range(0,len(multi_item_list))])
        for j in range(0,len(A[i])):
            if abs(A[i][j]) < 1e-6:
                A[i][j] = 0
    b = [-1 * i for i in multi_needy_list]
    res = optimize.linprog(c, A, b)
    unexcept_products_list = [0 for i in range(0,len(item_data))]
    for i in range(0,len(multi_recipe_list)):
        multi_recipe_list[i] = abs(res.x[i] * multi_recipe_list[i])
        unexcept_products_list[multi_item_list[i]] += res.slack[i]
        require_list = require_list + multi_recipe_list[i]
    return require_list,unexcept_products_list
    

# 原胚使用说明
## 函数说明
在第二个代码块中详细介绍了计算器参数设置相关的代码，包括单独/批量更改配方、保存/读取方案、自定义增产剂喷涂时的被喷涂等级等。目前常用的有以下几种：


change_product_mode_all(x)：
将所有能以x模式增产的配方的增产模式设置为x（x∈{0,1,2}，0为不增产，1为增产，2为加速）



change_recipe(物品名字，物品所选用的配方，配方所使用增产剂等级，配方增产模式)：
将“物品名字”的物品来源配方设置为其可选来源配方表中的对应值，并以输入的后两个参数作为增产剂使用方式。不知道物品的来源配方表中有哪些配方可以利用下方将提到的get_recipe_table()函数。



get_recipe_table(物品名)：
返回物品的可选来源配方表，但不会输出，想要输出的话得print(get_recipe_table('物品名'))。如：print(get_recipe_table('氢'))



proliferator_plus(增产剂名字，该增产剂使用时先被几级增产剂喷涂)：
自定义增产剂喷涂时的被喷涂等级，比如想要让3级增产剂使用时自喷涂就proliferator_plus('增产剂MK.Ⅲ',3)，想要让3级增产剂使用前先被2级增产剂喷涂就proliferator_plus('增产剂MK.Ⅲ',2)，想让这个喷涂3级增产剂的2级增产剂使用前被1级增产剂喷涂就proliferator_plus('增产剂MK.Ⅱ',3)。



set_needy_list(需求的物品名字，需求数量)：
设置“需求的物品名字”对应的物品中的需求量为“需求数量”。或者你也可以通过输入“needy_list[item_data['需求物品的名字'][0]] = 需求的数量”来实现。



save/load_data(文件名字)：
将你的数据保存到当前文件夹下对应名字的路径中去/从对应的路径中读取

## 使用说明
在下方的代码块设置好参数之后直接从头到尾按顺序运行所有代码块即可，如果之前已经运行过此说明之前的三个代码块了之后每次更改设置或是要重新运行时只需运行这下面的两个代码块即可。计算器的输出结果在从这往下数的第二个代码块，会告诉你所有产物的需求列表和多余产物的列表。

# 罗马数字的ⅠⅡⅢ列于此处方便使用者复制



## 下面一个代码块即使预先存在的参数设置的例子，目前的设置为全增产全高效的320w白糖需求列表


# 注：由于浮点误差问题还比较严重，尤其是当你选用大产量配方时浮点误差也会被随之放大，所以使用的时候要注意哪些部分是误差
# 如果第一个代码块报错那就看看注释，把！pip install 的注释前的#号删掉

In [647]:
# 参数调整部分
proliferator_plus('增产剂MK.Ⅲ',3)
change_product_mode_all(2)
change_product_mode_all(1)
change_recipe('氢',1,3,1)
change_recipe('重氢',1,3,1)
change_recipe('石墨烯',2,3,1)
change_recipe('卡西米尔晶体',2,3,1)
change_recipe('碳纳米管',2,3,1)
change_recipe('粒子容器',2,3,1)
change_recipe('金刚石',1,3,1)
change_recipe('高能石墨',1,3,1)
change_recipe('有机晶体',1,3,1)
change_recipe('硅石',1,3,1)
change_recipe('晶格硅',1,3,1)
change_recipe('精炼油',1,3,1)
change_recipe('临界光子',1,3,1)
print(get_recipe_table('氢'))
print(recipe_data[item_data['临界光子'][recipe_choices[item_data['临界光子'][0]]['来源配方']]])

# 需求列表设置部分
needy_list[item_data['石墨烯'][0]] = 0
needy_list[item_data['铁块'][0]] = 0
needy_list[item_data['氢'][0]] = 0
needy_list[item_data['制造台MK.Ⅱ'][0]] = 0
needy_list[item_data['宇宙矩阵'][0]] = 3200000
print(needy_list)

[{'原料': {}, '产物': {'氢': 1}, '设施': '轨道采集器', '时间': 0.125, '增产': 0}, {'原料': {'临界光子': 2}, '产物': {'反物质': 2, '氢': 2}, '设施': '粒子对撞机', '时间': 2, '增产': 2}, {'原料': {'精炼油': 1, '氢': 2}, '产物': {'高能石墨': 1, '氢': 3}, '设施': '精炼设备', '时间': 4, '增产': 2}, {'原料': {'原油': 2}, '产物': {'精炼油': 2, '氢': 1}, '设施': '精炼设备', '时间': 4, '增产': 3}, {'原料': {'可燃冰': 2}, '产物': {'石墨烯': 2, '氢': 1}, '设施': '化工设备', '时间': 2, '增产': 3}]
{'原料': {}, '产物': {'临界光子': 1}, '设施': '射线接收站', '时间': 10, '增产': 0}
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 3200000 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [648]:
# 计算执行部分——计算器本体
price_mat,multi_item_list,multi_recipe_list1,recipe_mat = price_mat_calculate()
require_list,unexcept_products_list = require_calculate(needy_list,price_mat,multi_item_list,multi_recipe_list1)
products_list1 = [0 for i in range(0,len(item_data))]
for i in range(0,len(item_data)):
    products_list[i] = require_list[i] * recipe_mat[i][i]
print('产物列表：')
show_require(products_list)
print('多余产物：')
show_require(unexcept_products_list)

产物列表：
{'铁矿': 9542041.60000027, '铜矿': 11796480.000000063, '石矿': 3355443.200000009, '煤矿': 17163306.72060712, '硅石': 23488102.40000002, '钛石': 11219763.200000057, '水': 2097152.0000000023, '原油': 2097152.0000006943, '氢': 16678912.000334933, '重氢': 6553599.999999992, '反物质': 2560000.0000030673, '可燃冰': 1677721.6000029997, '金伯利矿石': 4258058.68015177, '分形硅石': 1310720.0000000112, '光栅石': 8388608.000000004, '有机晶体': 1638400.0000000007, '刺笋结晶': 15844840.050569147, '单极磁石': 10485760.000000123, '临界光子': 2560000.000003067, '铁块': 10289152.00000015, '铜块': 14745600.000000078, '高纯硅块': 14680064.000000013, '钛块': 7012352.000000035, '高能石墨': 5406720.000000017, '玻璃': 2097152.0000000265, '磁铁': 1638400.000000098, '金刚石': 10645146.700379424, '晶格硅': 3276800.000000028, '精炼油': 2621440.0000008675, '塑料': 1638400.0000000002, '石墨烯': 2097152.0000037495, '碳纳米管': 6602016.68773715, '电路板': 11223040.0, '磁线圈': 2048000.0000001485, '微晶元件': 9175039.999999996, '钛晶石': 2048000.000000001, '处理器': 5734399.999999995, '粒子宽带': 2048000.0000000005, '

In [640]:
show_require(np.array(price_mat[item_data['氢'][0]])[0])
show_require(recipe_mat[item_data['氢'][0]])

{'氢': 0.9999999999999996}
{'原油': -1.6, '氢': 1.0, '精炼油': 2.0, '增产剂MK.Ⅲ': -0.021621621621621623}


In [641]:
save_data('测试1')
load_data('测试1')